In [1]:
import json
import pandas as pd
from tqdm.notebook import tqdm
import re

In [2]:
def show(x):
    print(x.shape)
    return x.head()

In [3]:
with open("../data_raw/science_parse_output.json") as file:
    lines = [json.loads(l) for l in tqdm(file.readlines())]
len(lines)

  0%|          | 0/1365 [00:00<?, ?it/s]

1365

In [4]:
lines[0]

{'name': '/home/jarobyte/Zotero/storage/FJGSUPN6/GCR-Powerpoint-Webinar-Jan-2020.pdf',
 'metadata': {'source': 'CRF',
  'title': None,
  'authors': ['Mimi McClure',
   'Laura Regassa',
   'Mike Roco',
   'Nora Savage',
   'Steven Breckler',
   'Toby Parcel'],
  'emails': ['gcrelig@nsf.gov.'],
  'sections': [{'heading': None,
    'text': 'NSF Big Idea: Growing Convergence Research\nNSF Big Idea: Growing Convergence Research\nWebinar Objectives\n• Overview of the program & program goals • Solicitation specifics • Review criteria • Q/A\nSubmit questions via the Q/A panel in Zoom or via email to gcrelig@nsf.gov.\nNSF Big Idea: Growing Convergence Research\nConvergence at NSF\n• Convergence is a multifaceted concept • It has long history at NSF; part of NSF’s portfolio since 1954 • It is currently an important component of several parts of NSF’s\nportfolio, including • the 6 research-oriented Big Ideas • NSF Convergence Accelerator • Growing Convergence Research (GCR) a process-oriented Big

In [5]:
raw = pd.DataFrame(lines)\
.assign(title = lambda df: df.metadata.map(lambda x: x["title"]),
        abstract = lambda df: df.metadata.map(lambda x: x["abstractText"]),
        sections = lambda df: df.metadata.map(lambda x: x["sections"]),
        references = lambda df: df.metadata.map(lambda x: x["references"]),
        citations = lambda df: df.metadata.map(lambda x: x["referenceMentions"]),
)\

show(raw)

(1365, 7)


,name,metadata,title,abstract,sections,references,citations
0,/home/jarobyte/Zotero/storage/FJGSUPN6/GCR-Pow...,"{'source': 'CRF', 'title': None, 'authors': ['...",None,None,"[{'heading': None, 'text': 'NSF Big Idea: Grow...",[],[]
1,/home/jarobyte/Zotero/storage/E47J4XWS/Beineke...,"{'source': 'CRF', 'title': 'Characterizations ...",Characterizations of Derived Graphs,The derived graph of a graph G has the edges o...,None,[{'title': 'On the Uniqueness of the Triangula...,"[{'referenceID': 0, 'context': 'JOURNAL OF COM..."
2,/home/jarobyte/Zotero/storage/K9SMVNFH/IDST at...,"{'source': 'META', 'title': 'IDST at TREC 2019...",IDST at TREC 2019 Deep Learning Track: Deep Ca...,This paper describes our participation in the ...,"[{'heading': None, 'text': 'KEYWORDS cascade r...",[{'title': 'Overview of the TREC 2019 deep lea...,"[{'referenceID': 1, 'context': 'Our approach i..."
3,/home/jarobyte/Zotero/storage/FEFDDM7K/Kirsch ...,"{'source': 'CRF', 'title': 'BatchBALD: Efficie...",BatchBALD: Efficient and Diverse Batch Acquisi...,"We develop BatchBALD, a tractable approximatio...","[{'heading': '1 Introduction', 'text': 'A key ...",[{'title': 'Toward the next generation of reco...,"[{'referenceID': 6, 'context': 'Active learnin..."
4,/home/jarobyte/Zotero/storage/GXE9CYPA/acl13-S...,"{'source': 'META', 'title': 'A Sentence Compre...",A Sentence Compression Based Framework to Quer...,We consider the problem of using sentence comp...,"[{'heading': None, 'text': 'Proceedings of the...",[{'title': 'Syntax directed translations and t...,"[{'referenceID': 18, 'context': 'specific or n..."


In [6]:
raw_na = raw.where(raw.applymap(lambda x: x is not None and x != [])).dropna().reset_index(drop = True)
show(raw_na)

(1091, 7)


,name,metadata,title,abstract,sections,references,citations
0,/home/jarobyte/Zotero/storage/K9SMVNFH/IDST at...,"{'source': 'META', 'title': 'IDST at TREC 2019...",IDST at TREC 2019 Deep Learning Track: Deep Ca...,This paper describes our participation in the ...,"[{'heading': None, 'text': 'KEYWORDS cascade r...",[{'title': 'Overview of the TREC 2019 deep lea...,"[{'referenceID': 1, 'context': 'Our approach i..."
1,/home/jarobyte/Zotero/storage/FEFDDM7K/Kirsch ...,"{'source': 'CRF', 'title': 'BatchBALD: Efficie...",BatchBALD: Efficient and Diverse Batch Acquisi...,"We develop BatchBALD, a tractable approximatio...","[{'heading': '1 Introduction', 'text': 'A key ...",[{'title': 'Toward the next generation of reco...,"[{'referenceID': 6, 'context': 'Active learnin..."
2,/home/jarobyte/Zotero/storage/GXE9CYPA/acl13-S...,"{'source': 'META', 'title': 'A Sentence Compre...",A Sentence Compression Based Framework to Quer...,We consider the problem of using sentence comp...,"[{'heading': None, 'text': 'Proceedings of the...",[{'title': 'Syntax directed translations and t...,"[{'referenceID': 18, 'context': 'specific or n..."
3,/home/jarobyte/Zotero/storage/AYBCIJ3E/Ghaeini...,"{'source': 'CRF', 'title': 'DR-BiLSTM: Depende...",DR-BiLSTM: Dependent Reading Bidirectional LST...,We present a novel deep learning architecture ...,"[{'heading': '1 Introduction', 'text': 'Natura...",[{'title': 'Neural machine translation by join...,"[{'referenceID': 10, 'context': 'Recognizing T..."
4,/home/jarobyte/Zotero/storage/W98Z72WR/Yang et...,"{'source': 'CRF', 'title': 'On Minimizing Cost...",On Minimizing Cost in Legal Document ReviewWor...,Technology-assisted review (TAR) refers to hum...,"[{'heading': None, 'text': 'CCS CONCEPTS • Inf...",[{'title': 'Towards minimizing the annotation ...,"[{'referenceID': 1, 'context': 'enterprise doc..."


In [7]:
papers = raw_na[["title", "abstract", "sections"]].reset_index().rename(columns = {"index":"paper_id"})
show(papers)

(1091, 4)


,paper_id,title,abstract,sections
0,0,IDST at TREC 2019 Deep Learning Track: Deep Ca...,This paper describes our participation in the ...,"[{'heading': None, 'text': 'KEYWORDS cascade r..."
1,1,BatchBALD: Efficient and Diverse Batch Acquisi...,"We develop BatchBALD, a tractable approximatio...","[{'heading': '1 Introduction', 'text': 'A key ..."
2,2,A Sentence Compression Based Framework to Quer...,We consider the problem of using sentence comp...,"[{'heading': None, 'text': 'Proceedings of the..."
3,3,DR-BiLSTM: Dependent Reading Bidirectional LST...,We present a novel deep learning architecture ...,"[{'heading': '1 Introduction', 'text': 'Natura..."
4,4,On Minimizing Cost in Legal Document ReviewWor...,Technology-assisted review (TAR) refers to hum...,"[{'heading': None, 'text': 'CCS CONCEPTS • Inf..."


In [8]:
raw_na.citations.map(len, na_action = "ignore").sum()

56746

In [9]:
citations_raw = raw_na.citations.dropna().explode()
show(citations_raw)

(56746,)


0    {'referenceID': 1, 'context': 'Our approach is...
0    {'referenceID': 8, 'context': 'Different from ...
0    {'referenceID': 5, 'context': 'Moreover, for f...
0    {'referenceID': 6, 'context': 'The proposed mo...
0    {'referenceID': 6, 'context': 'Following [7], ...
Name: citations, dtype: object

In [10]:
citations = pd.DataFrame(citations_raw.tolist())\
.assign(paper_id = citations_raw.index)\
[["paper_id", "referenceID", "context", "startOffset", "endOffset"]]\
.rename(columns = {"referenceID":"internal_reference_id", 
                   "startOffset":"start_offset", 
                   "endOffset":"end_offset"})

show(citations)
citations.head(20)

(56746, 5)


,paper_id,internal_reference_id,context,start_offset,end_offset
0,0,1,Our approach is mainly based on the BERT langu...,56,59
1,0,8,Different from many other ranking methods whic...,216,219
2,0,5,"Moreover, for full ranking subtask, we use a s...",78,81
3,0,6,The proposed model is based on the pointer-gen...,59,62
4,0,6,"Following [7], we also use a soft switch to ch...",10,13
5,0,6,where pдen is a soft switch used in [7].,36,39
6,0,9,"After the query generation model is trained, f...",147,151
7,0,5,The expanded passages are then indexed and we ...,100,103
8,0,10,"For the BM25 search engine, we use the off-the...",85,89
9,0,8,"Inspired from StructBERT [9], we first modify ...",25,28


In [11]:
citations.context[2]

'Moreover, for full ranking subtask, we use a simple and effective BM25 method [6] to retrieve a collection of candidate passages, by leveraging the document expansion technique.'

In [12]:
references = raw_na.references.explode().reset_index()\
.assign(internal_reference_id = lambda df: df.groupby("index").cumcount(),
        title = lambda df: df.references.map(lambda x: x["title"], na_action = "ignore"))\
.assign(title_join = lambda df: df.title.str.replace("[^a-zA-Z]", "").str.lower())\
.rename(columns = {"index":"paper_id"})


show(references)


(44380, 5)


/tmp/ipykernel_13227/969181190.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  .assign(title_join = lambda df: df.title.str.replace("[^a-zA-Z]", "").str.lower())\


,paper_id,references,internal_reference_id,title,title_join
0,0,{'title': 'Overview of the TREC 2019 deep lear...,0,Overview of the TREC 2019 deep learning track,overviewofthetrecdeeplearningtrack
1,0,{'title': 'Bert: Pre-training of deep bidirect...,1,Bert: Pre-training of deep bidirectional trans...,bertpretrainingofdeepbidirectionaltransformers...
2,0,{'title': 'ALBERT: A lite BERT for self-superv...,2,ALBERT: A lite BERT for self-supervised learni...,albertalitebertforselfsupervisedlearningoflang...
3,0,{'title': 'Roberta: A robustly optimized bert ...,3,Roberta: A robustly optimized bert pretraining...,robertaarobustlyoptimizedbertpretrainingapproach
4,0,"{'title': 'Passage Re-ranking with BERT', 'aut...",4,Passage Re-ranking with BERT,passagererankingwithbert


In [13]:
unique_references = references.groupby("title_join")\
.agg(["min", "count"])\
.title\
.sort_values("count", ascending = False)\
.reset_index()\
.reset_index()\
.rename(columns = {"index":"global_reference_id"})

show(unique_references)

(24732, 4)


,global_reference_id,title_join,min,count
0,0,distributedrepresentationsofwordsandphrasesand...,Distributed Representations of Words and Phras...,172
1,1,bertpretrainingofdeepbidirectionaltransformers...,BERT: Pre-training of Deep Bidirectional Trans...,167
2,2,attentionisallyouneed,Attention Is All You Need,155
3,3,gloveglobalvectorsforwordrepresentation,GloVe : Global Vectors for Word Representation,154
4,4,adamamethodforstochasticoptimization,Adam: A Method for Stochastic Optimization,124


In [14]:
final = citations.merge(references, how = "left", on = ["paper_id", "internal_reference_id"])\
.merge(unique_references, how = "left", on = "title_join")
show(final)

(56746, 11)


,paper_id,internal_reference_id,context,start_offset,end_offset,references,title,title_join,global_reference_id,min,count
0,0,1,Our approach is mainly based on the BERT langu...,56,59,{'title': 'Bert: Pre-training of deep bidirect...,Bert: Pre-training of deep bidirectional trans...,bertpretrainingofdeepbidirectionaltransformers...,1,BERT: Pre-training of Deep Bidirectional Trans...,167
1,0,8,Different from many other ranking methods whic...,216,219,{'title': 'StructBERT: Incorporating Language ...,StructBERT: Incorporating Language Structures ...,structbertincorporatinglanguagestructuresintop...,475,StructBERT: Incorporating Language Structures ...,9
2,0,5,"Moreover, for full ranking subtask, we use a s...",78,81,{'title': 'The probabilistic relevance framewo...,The probabilistic relevance framework: BM25 an...,theprobabilisticrelevanceframeworkbmandbeyond,155,The Probabilistic Relevance Framework: BM25 an...,16
3,0,6,The proposed model is based on the pointer-gen...,59,62,{'title': 'Get to the point: Summarization wit...,Get to the point: Summarization with pointer-g...,gettothepointsummarizationwithpointergenerator...,25,Get To The Point: Summarization with Pointer-G...,45
4,0,6,"Following [7], we also use a soft switch to ch...",10,13,{'title': 'Get to the point: Summarization wit...,Get to the point: Summarization with pointer-g...,gettothepointsummarizationwithpointergenerator...,25,Get To The Point: Summarization with Pointer-G...,45


# final datasets

In [15]:
final_citations = final[["paper_id", "internal_reference_id", "global_reference_id", "context", "start_offset", 
                         "end_offset"]]\
.dropna()\
.assign(global_reference_id = lambda df: df.global_reference_id.map(int, na_action = "ignore"))
show(final_citations)

(56746, 6)


,paper_id,internal_reference_id,global_reference_id,context,start_offset,end_offset
0,0,1,1,Our approach is mainly based on the BERT langu...,56,59
1,0,8,475,Different from many other ranking methods whic...,216,219
2,0,5,155,"Moreover, for full ranking subtask, we use a s...",78,81
3,0,6,25,The proposed model is based on the pointer-gen...,59,62
4,0,6,25,"Following [7], we also use a soft switch to ch...",10,13


In [16]:
final_papers = papers.assign(text = lambda df: df.sections.map(lambda x: " ".join([s["text"] for s in x]), 
                                                               na_action = "ignore"))\
.assign(title = lambda df: df.title.str.replace(r"\s+", " ").str.strip(),
        abstract = lambda df: df.abstract.str.replace(r"\s+", " ").str.strip(),
        text = lambda df: df.text.str.replace(r"\s+", " ").str.strip())
show(final_papers)

/tmp/ipykernel_13227/2210464430.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  .assign(title = lambda df: df.title.str.replace(r"\s+", " ").str.strip(),
/tmp/ipykernel_13227/2210464430.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  abstract = lambda df: df.abstract.str.replace(r"\s+", " ").str.strip(),
/tmp/ipykernel_13227/2210464430.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  text = lambda df: df.text.str.replace(r"\s+", " ").str.strip())


(1091, 5)


,paper_id,title,abstract,sections,text
0,0,IDST at TREC 2019 Deep Learning Track: Deep Ca...,This paper describes our participation in the ...,"[{'heading': None, 'text': 'KEYWORDS cascade r...","KEYWORDS cascade ranking, pre-trained language..."
1,1,BatchBALD: Efficient and Diverse Batch Acquisi...,"We develop BatchBALD, a tractable approximatio...","[{'heading': '1 Introduction', 'text': 'A key ...",A key problem in deep learning is data efficie...
2,2,A Sentence Compression Based Framework to Quer...,We consider the problem of using sentence comp...,"[{'heading': None, 'text': 'Proceedings of the...",Proceedings of the 51st Annual Meeting of the ...
3,3,DR-BiLSTM: Dependent Reading Bidirectional LST...,We present a novel deep learning architecture ...,"[{'heading': '1 Introduction', 'text': 'Natura...",Natural Language Inference (NLI; a.k.a. Recogn...
4,4,On Minimizing Cost in Legal Document ReviewWor...,Technology-assisted review (TAR) refers to hum...,"[{'heading': None, 'text': 'CCS CONCEPTS • Inf...",CCS CONCEPTS • Information systems → Informati...


In [17]:
references_abstract = unique_references\
.merge(final_papers.assign(title_join = lambda df: df.title.str.replace("[^a-zA-Z]", "").str.lower()),
       how = "left")\
.drop(columns = "title")\
.rename(columns = {"min":"title"})\
[["global_reference_id", "title", "abstract", "title_join"]]\
.drop_duplicates()

show(references_abstract)

(24743, 4)


/tmp/ipykernel_13227/484812439.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  .merge(final_papers.assign(title_join = lambda df: df.title.str.replace("[^a-zA-Z]", "").str.lower()),


,global_reference_id,title,abstract,title_join
0,0,Distributed Representations of Words and Phras...,The recently introduced continuous Skip-gram m...,distributedrepresentationsofwordsandphrasesand...
1,1,BERT: Pre-training of Deep Bidirectional Trans...,We introduce a new language representation mod...,bertpretrainingofdeepbidirectionaltransformers...
2,1,BERT: Pre-training of Deep Bidirectional Trans...,We introduce a new language representation mod...,bertpretrainingofdeepbidirectionaltransformers...
6,2,Attention Is All You Need,The dominant sequence transduction models are ...,attentionisallyouneed
7,2,Attention Is All You Need,The dominant sequence transduction models are ...,attentionisallyouneed


# complete references from the arxiv dataset

In [ ]:
titles = []
abstracts = []
with open("../data_raw/arxiv-metadata-oai-snapshot.json") as file:
    for l in tqdm(file):
        d = json.loads(l)
        titles.append(re.sub(r"[^a-zA-Z]", "",d["title"]).lower())
        abstracts.append(d["abstract"])

0it [00:00, ?it/s]

In [ ]:
arxiv = pd.DataFrame(dict(title_join = titles, abstract = abstracts))
show(arxiv)

In [ ]:
full_references = references_abstract.merge(arxiv, on = "title_join", how = "left")\
.assign(abstract = lambda df: df.abstract_y.combine_first(df.abstract_x))\
.drop(columns = ["abstract_x", "title_join", "abstract_y"])\
.drop_duplicates()\
.assign(title = lambda df: df.title.str.replace(r"\s+", " ").str.strip(),
        abstract = lambda df: df.abstract.str.replace(r"\s+", " ").str.strip())

show(full_references)
full_references.head(20)

# paper sections

In [ ]:
paper_sections = final_papers\
.drop(columns = ["title", "text", "abstract"])\
.explode("sections")\
.assign(section_order = lambda df: df.groupby("paper_id").cumcount(),
        heading = lambda df: df.sections.map(lambda x: x["heading"], na_action = "ignore"),
        text = lambda df: df.sections.map(lambda x: x["text"], na_action = "ignore"))\
.drop(columns = "sections")\
.assign(heading = lambda df: df.heading.str.replace(r"\s+", " ").str.strip(),
        text = lambda df: df.text.str.replace(r"\s+", " ").str.strip())

show(paper_sections)

# writing to disk

In [ ]:
final_citations.to_csv("../data/citations.csv", index = False)

In [ ]:
final_papers.drop(columns = "sections").to_csv("../data/papers.csv", errors = "replace", index = False)

In [ ]:
full_references.to_csv("../data/references.csv", index = False)

In [ ]:
paper_sections.to_csv("../data/paper_sections.csv", index = False, errors = "replace")